Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Onnx Pipeline

This repository shows how to deploy and use Onnx pipeline with dockers including convert model, generate input and performance test.

# Prerequisites

Pull dockers from Azure. It should take several minutes.

In [1]:
!sh build.sh

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Error response from daemon: Get https://ziylregistry.azurecr.io/v2/: unauthorized: authentication required
Using default tag: latest
latest: Pulling from onnx-converter
Digest: sha256:43036294bac2bc2c88e5a42ff85a5cd38ac966004b02e4a25ca54f83ca970010
Status: Image is up to date for ziylregistry.azurecr.io/onnx-converter:latest
Using default tag: latest
latest: Pulling from perf_test
Digest: sha256:0b93e6a1d4e4cd5e0057cf503fce53b8702d2445252b7837e844e50752d2a369
Status: Image is up to date for ziylregistry.azurecr.io/perf_test:latest


Install the onnxpipeline SDK

In [1]:
import onnxpipeline

# test tensorflow
pipeline = onnxpipeline.Pipeline('mnist/model')

# test pytorch
#pipeline = onnxpipeline.Pipeline('pytorch')

# cntk 
#pipeline = onnxpipeline.Pipeline('cntk')

# keras
#pipeline = onnxpipeline.Pipeline('KerasToONNX')

# sklearn
#pipeline = onnxpipeline.Pipeline('sklearn')

# caffe
#pipeline = onnxpipeline.Pipeline('caffe')

# empty
#pipeline = onnxpipeline.Pipeline()

# test mxnet fail
#pipeline = onnxpipeline.Pipeline('mxnet')


## Run parameters

(1) local_directory: string

    Required. The path of local directory where would be mounted to the docker. All operations will be executed from this path.

(2) mount_path: string

    Optional. The path where the local_directory will be mounted in the docker. Default is "/mnt/model".

(3) print_logs: boolean

    Optional. Whether print the logs from the docker. Default is True.


# Config information for ONNX pipeline

In [2]:
pipeline.config()

-----------config----------------
           Container information: <docker.client.DockerClient object at 0x7f5e940ca550>
 Local directory path for volume: /home/chuche/notebook/mnist/model
Volume directory path in dockers: /mnt/model
                     Result path: result
        Converted directory path: test
        Converted model filename: model.onnx
            Converted model path: test/model.onnx
        Print logs in the docker: True


# Convert model to ONNX

This image is used to convert model from major model frameworks to onnx. Supported frameworks are - caffe, cntk, coreml, keras, libsvm, mxnet, scikit-learn, tensorflow and pytorch.


You can run the docker image with customized parameters.

In [3]:
 # test tensorflow
model = pipeline.convert_model(model_type='tensorflow')

# test pytorch
#model = pipeline.convert_model(model_type='pytorch', model='saved_model.pb', input_json='input.json', convert_json=True)

# test cntk
#model = pipeline.convert_model(model_type='cntk', model='ResNet50_ImageNet_Caffe.model')

# test keras
#model = pipeline.convert_model(model_type='keras', model='keras_Average_ImageNet.keras', input_json='input.json', convert_json=True)

# test sklearn
#model = pipeline.convert_model(model_type='scikit-learn', model='sklearn_svc.joblib', initial_types=("float_input", "FloatTensorType([1,4])"), input_json='input.json', convert_json=True)

# test caffe
#model = pipeline.convert_model(model_type='caffe', model='bvlc_alexnet.caffemodel', caffe_model_prototxt ='deploy.prototxt', input_json='input.json', convert_json=True)


# test mxnet
#model = pipeline.convert_model(model_type='mxnet', model='resnet.json', model_params='resnet.params', model_input_shapes='(1,3,224,224)')

2019-06-19 00:14:39,321 - INFO - Using tensorflow=1.12.0, onnx=1.5.0, tf2onnx=1.5.1/0c735a

2019-06-19 00:14:39,321 - INFO - Using opset <onnx, 7>

2019-06-19 00:14:39,446 - INFO - 

2019-06-19 00:14:39,481 - INFO - Optimizing ONNX model

2019-06-19 00:14:39,515 - INFO - After optimization: Add -2 (4->2), Const +1 (12->13), Gather +1 (0->1), Identity -5 (5->0), Transpose -6 (8->2)

2019-06-19 00:14:39,519 - INFO - 

2019-06-19 00:14:39,519 - INFO - Successfully converted TensorFlow model /mnt/model/ to ONNX

2019-06-19 00:14:39,539 - INFO - ONNX model is saved at /mnt/model/test/model.onnx

2019-06-19 00:14:39.880335: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA



-------------

Model Conversion





-------------

MODEL INPUT GENERATION(if needed)



Input.pb already exists. Skipping dummy input generation. 



-------------

MODEL CORRECTNESS VERIFICATION





Check the ONNX 

## Run parameters

(1) model: string

    Required. The path of the model that needs to be converted.

(2) output_onnx_path: string

    Required. The path to store the converted onnx model. Should end with ".onnx". e.g. output.onnx

(3) model_type: string

    Required. The name of original model framework. Available types are caffe, cntk, coreml, keras, libsvm, mxnet, scikit-learn, tensorflow and pytorch.

(4) model_inputs: string

    (tensorflow) Optional. The model's input names. Required for tensorflow frozen models and checkpoints.

(5) model_outputs: string

    (tensorflow) Optional. The model's output names. Required for tensorflow frozen models checkpoints.

(6) model_params: string 

    (mxnet) Optional. The params of the model if needed.

(7) model_input_shapes: list of tuple 

    (pytorch, mxnet) Optional. List of tuples. The input shape(s) of the model. Each dimension separated by ','.

(8) target_opset: int

    Optional. Specifies the opset for ONNX, for example, 7 for ONNX 1.2, and 8 for ONNX 1.3. Defaults to 7.
    
(9) caffe_model_prototxt: string

    (caffe) Optional. The filename of deploy prototxt for the caffe madel. 

(10) initial_types: tuple (string, string)

    (scikit-learn) Optional. A tuple consist two strings. The first is data type and the second is the size of tensor type e.g., ('float_input', 'FloatTensorType([1,4])')

(11) input_json: string

    Optional. Use JSON file as input parameters.

(12) convert_json: boolean
    
    Optional. Convert all parameters into JSON file for input parameters.


# Performance test tool

You can run perf_test using command python perf_test.py [Your model path] [Output path on the docker]. You can use the same arguments as for onnxruntime_pert_test tool, e.g. -m for mode, -e to specify execution provider etc. By default it will try all providers available.

In [5]:
result = pipeline.perf_test(model=model, r=99)
#result = pipeline.perf_test()   # is ok, too

Cores:  6



OMP_WAIT_POLICY=PASSIVE

OMP_NUM_THREADS=1

/perf_test/bin/RelWithDebInfo/cpu_openmp/onnxruntime_perf_test -m times -r 99 /mnt/model/test/model.onnx /mnt/model/result/a9b68254-ff04-4eed-bf27-82421701bb29

cpu_openmp_OMP_NUM_THREADS_1_passive 0.001039823

lower: 1, mid: 2, upper: 3

Traceback (most recent call last):

  File "/perf_test/perf_test.py", line 473, in <module>

    json_record["cpu_usage"] = test.cpu / 100

AttributeError: 'PerfTestParams' object has no attribute 'cpu'



OMP_WAIT_POLICY=PASSIVE

OMP_NUM_THREADS=2

/perf_test/bin/RelWithDebInfo/cpu_openmp/onnxruntime_perf_test -m times -r 99 /mnt/model/test/model.onnx /mnt/model/result/3af7c056-165e-4db8-9532-54a7c6881325

cpu_openmp_OMP_NUM_THREADS_2_passive 0.000748495

current latency:  0.000748495

best latency:  0.001039823



OMP_WAIT_POLICY=PASSIVE

OMP_NUM_THREADS=4

/perf_test/bin/RelWithDebInfo/cpu_openmp/onnxruntime_perf_test -m times -r 99 /mnt/model/test/model.onnx /mnt/model/result/3c9d7e98-8f3e-4

# Run parameters

(1) model: string

    Required. The path of the model that wants to be performed.
    
(2) result: string

    Optional. The path of the result.
    
(3) config: string (choices=["Debug", "MinSizeRel", "Release", "RelWithDebInfo"])

    Optional. Configuration to run. Default is "RelWithDebInfo".
    
(4) m: string (choices=["duration", "times"])

    Optional. Specifies the test mode. Value could be 'duration' or 'times'. Default is "times".

(5) e: string (choices=["cpu", "cuda", "mkldnn"])

    Optional. help="Specifies the provider 'cpu','cuda','mkldnn'. Default is ''.
    
(6) r: string

    Optional. Specifies the repeated times if running in 'times' test mode. Default:20.
    
(7) t: string

    Optional. Specifies the seconds to run for 'duration' mode. Default:10.
    
(8) x: string

    Optional. Use parallel executor, default (without -x): sequential executor.
    
(9) n: string

    Optional. OMP_NUM_THREADS value.
    
(10) s: string

    Optional. Show percentiles for top n runs. Default:5.
   

In [ ]:
pipeline.print_result(result)
#pipeline.print_result() # is ok, too

# Run parameters

(1) result: string
Optional. The path of the result.

# netron

In [3]:
# only workable for notebook in the local server 
import netron
netron.start(model) # 'model.onnx'
from IPython.display import IFrame
IFrame('http://localhost:8080', width=700, height=350)


Stopping http://localhost:8080
Serving 'model.onnx' at http://localhost:8080
